In [1]:
import pandas as pd
df_court = pd.read_csv(r'C:\courses\DataManagement and Vis\project\Municipal_Court_Caseload_Information_FY_2023.csv')
print(df_court.head())

  Offense Case Type Offense Date Offense Time Violation Charge Code  \
0                NT    10/1/2023      0:26:00                 60110   
1                TR    10/1/2023     13:40:00                   710   
2                TR    10/1/2023     16:39:00                 32210   
3                TR    10/1/2023     17:04:00                   610   
4                OR    10/1/2023     10:26:00                 64111   

                Offense Charge Description       Offense Street Name  \
0                      Public Intoxication            4905 TERI ROAD   
1                            Ran Red Light       4010 SOUTHWEST PKWY   
2  Crossing Property To Turn Right Or Left         2414 S LAMAR BLVD   
3                            Ran Stop Sign          BLUE CREST DRIVE   
4                        Animal - At Large  6600 BLOCK ASHLAND DRIVE   

  Offense Cross Street  School Zone  Construction Zone Case Closed   Race  \
0                  NaN        False              False        T

In [2]:
df_street_zip = pd.read_csv(r'C:\courses\DataManagement and Vis\project\austin_street_to_zip_mapping.csv')
print(df_street_zip.head())

             STREET  POSTCODE    CITY
0  DOYLE OVERTON RD     78719  Austin
1           MAHA RD     78719  Austin
2       POCMONT TRL     78719  Austin
3         EVELYN RD     78747  Austin
4    S SH  45 E  WB     78747  Austin


In [3]:
import re

replacements =  {
    "ROAD": "RD", "RD": "RD",
    "STREET": "ST", "ST": "ST",
    "AVENUE": "AVE", "AVE": "AVE",
    "DRIVE": "DR", "DR": "DR",
    "COURT": "CT", "CT": "CT",
    "LANE": "LN", "LN": "LN",
    "CIRCLE": "CIR", "CIR": "CIR",
    "TRAIL": "TRL", "TRL": "TRL",
    "PARKWAY": "PKWY", "PKWY": "PKWY",
    "HIGHWAY": "HWY", "HWY": "HWY",
    "PLACE": "PL", "PL": "PL",
    "PATH": "PATH",
    "WAY": "WAY",
    "LOOP": "LOOP",
    "COVE": "CV", "CV": "CV",

    # directions (important!)
    "NORTH": "N", "N": "N",
    "SOUTH": "S", "S": "S",
    "EAST": "E", "E": "E",
    "WEST": "W", "W": "W",
    "NORTHEAST": "NE", "NE": "NE",
    "NORTHWEST": "NW", "NW": "NW",
    "SOUTHEAST": "SE", "SE": "SE",
    "SOUTHWEST": "SW", "SW": "SW",

    # lane directions
    "NB": "NB", "SB": "SB", "EB": "EB", "WB": "WB"
}

direction_garbage = {
   "BLOCK", "NB", "SB", "EB", "WB", "NORTHBOUND", "SOUTHBOUND", "EASTBOUND", "WESTBOUND"
}
ordinals = {
    "FIRST": "1ST", "SECOND": "2ND", "THIRD": "3RD", "FOURTH": "4TH",
    "FIFTH": "5TH", "SIXTH": "6TH", "SEVENTH": "7TH", "EIGHTH": "8TH",
    "NINTH": "9TH", "TENTH": "10TH"
}

def clean_and_normalize(s):
    if pd.isna(s) or s == 'nan':
        return s

    s = s.upper()

    # deleting numbers in first position with a space 
    s = re.sub(r"^\d+\s+", "", s)

    # deleting punctuations
    s = re.sub(r"[^\w\s]", "", s)

    tokens = s.split()
    clean_tokens = []

    for i, t in enumerate(tokens):
        # deleting noises (NB, SB, etc)
        if t in direction_garbage:
            continue
        #FIRST -> 1ST
        t = ordinals.get(t, t)

        # correctint the abbrevations (ROAD -> RD)
        t = replacements.get(t, t)

        clean_tokens.append(t)

    return " ".join(clean_tokens).strip()

##Clean and normalize main dataset's street name Municipal_Court

In [4]:
df_court["street_clean1"] = df_court["Offense Street Name"].apply(clean_and_normalize)

count_before1 = len(df_court)
#removing duplicated streets
df1_unique = df_court.drop_duplicates(subset=["street_clean1"])
count_after1 = len(df1_unique)

print(f"Total number of rows in first dataset Municipal_Court: {count_before1}")
print(f"Number of rows after deleting duplicated names: {count_after1}")

Total number of rows in first dataset Municipal_Court: 225624
Number of rows after deleting duplicated names: 5449


In [6]:
df1_unique['street_clean1'].to_csv('df1_unique_streets_only.csv', index=False)

In [5]:
df1_unique_streets_only = pd.read_csv(r'C:\Users\Utente\df1_unique_streets_only.csv')
print(df1_unique_streets_only.head(10))

      street_clean1
0           TERI RD
1           SW PKWY
2      S LAMAR BLVD
3     BLUE CREST DR
4        ASHLAND DR
5       W PARMER LN
6     LEVANDER LOOP
7           LAZY ON
8  WALSH TARLTON LN
9      S MOPAC EXPY


##Clean and normalize second dataset's street name austin_street

In [6]:
# Replace NaN with an empty string, then clean
df_street_zip["street_clean2"] = df_street_zip["STREET"].fillna("").astype(str).str.upper().str.strip()

# normalization function now receives "" instead of a float
df_street_zip["street_clean2"] = df_street_zip["street_clean2"].apply(clean_and_normalize)

In [7]:
count_before2 = len(df_street_zip)

#removing duplicated streets
df2_unique = df_street_zip.drop_duplicates(subset=["street_clean2", "POSTCODE"]) #there are equal street names with diffretnt zipcode, so they are not unique place

count_after2 = len(df2_unique)

print(f"Total number of rows in second dataset austin_street: {count_before2}")
print(f"Number of rows after deleting duplicated names: {count_after2}")

Total number of rows in second dataset austin_street: 11840
Number of rows after deleting duplicated names: 11773


In [10]:
df2_unique[['street_clean2', 'POSTCODE']].to_csv('df2_unique_streets_zipcode.csv', index=False)

In [8]:
df2_unique_streets_zipcode = pd.read_csv(r'C:\Users\Utente\df2_unique_streets_zipcode.csv')
print(df2_unique_streets_zipcode.head(10))

      street_clean2  POSTCODE
0  DOYLE OVERTON RD     78719
1           MAHA RD     78719
2       POCMONT TRL     78719
3         EVELYN RD     78747
4         S SH 45 E     78747
5     S SH 130 SVRD     78719
6      MAHA LOOP RD     78719
7      S US 183 HWY     78719
8      S US 183 HWY     78747
9    TOM SASSMAN RD     78747


##to find how many unique cities matches in 2 different datasets

In [9]:
#Integerating 2 datasets
common = set(df1_unique_streets_only["street_clean1"]) & set(df2_unique_streets_zipcode["street_clean2"])
# Remove 'nan' or empty strings if they aren't real cities -> Data Quality
common.discard('NAN') 
common.discard('')
common.discard('nan')
print("Matches:", len(common))

Matches: 1987


In [10]:
df_neigh = pd.read_csv(r'C:\courses\DataManagement and Vis\project\austin_zip_to_neighborhood_full.csv')
df_pop = pd.read_csv(r'C:\courses\DataManagement and Vis\project\austin_population_by_zip_scraped.csv')
valid_pop = df_pop[df_pop["Population"] > 0]

zip_neigh = set(df_neigh["ZIP_Code"])
zip_pop_valid = set(valid_pop["ZIP_Code"])
zip_street = set(df2_unique_streets_zipcode["POSTCODE"])

common_zips = zip_neigh & zip_pop_valid & zip_street

print("Common ZIPs:", len(common_zips))
common_zips

Common ZIPs: 40


{78701,
 78702,
 78703,
 78704,
 78705,
 78721,
 78722,
 78723,
 78724,
 78725,
 78726,
 78727,
 78728,
 78729,
 78730,
 78731,
 78732,
 78733,
 78735,
 78736,
 78737,
 78738,
 78739,
 78741,
 78742,
 78744,
 78745,
 78746,
 78747,
 78748,
 78749,
 78750,
 78751,
 78752,
 78753,
 78754,
 78756,
 78757,
 78758,
 78759}

In [11]:
# 1. Get the set of ZIP codes associated ONLY with the streets in 'common'
# We look at df2 to find the POSTCODE for every street that exists in 'common'
zips_from_matched_streets = set(df2_unique_streets_zipcode[
    df2_unique_streets_zipcode['street_clean2'].isin(common)
]['POSTCODE'])

# 2. Find the intersection between matched street ZIPs and valid pool (common_zips)
final_overlap = zips_from_matched_streets & common_zips

print(f"ZIP codes found in matched streets: {len(zips_from_matched_streets)}")
print(f"ZIP codes in your valid pool (Neighborhood/Pop/Street): {len(common_zips)}")
print(f"---")
print(f"Shared ZIP codes (Matches): {len(final_overlap)}")

ZIP codes found in matched streets: 43
ZIP codes in your valid pool (Neighborhood/Pop/Street): 40
---
Shared ZIP codes (Matches): 40


##Final cleaning

In [12]:
df_court['street_clean'] = df_court['Offense Street Name'].str.upper().str.strip()
df2_unique_streets_zipcode['street_clean2'] = df2_unique_streets_zipcode['street_clean2'].str.upper().str.strip()
df1_unique_streets_only['street_clean1'] = df1_unique_streets_only['street_clean1'].str.upper().str.strip()

In [13]:
crimes_with_zip = pd.merge(df_court, df2_unique_streets_zipcode
                           [['street_clean2', 'POSTCODE']], left_on='street_clean', right_on='street_clean2', how='left')
crimes_with_zip.rename(columns={'POSTCODE': 'ZIP_Code'}, inplace=True)

In [14]:
match_rate = crimes_with_zip['ZIP_Code'].notna().mean() * 100
print(f"Match rate (ZIP is found): {match_rate:.2f}%")

Match rate (ZIP is found): 1.93%


In [16]:
crimes_with_neigh = pd.merge(crimes_with_zip, df_neigh, on='ZIP_Code', how='left')
valid_pop = df_pop[df_pop['Population'] > 0][['ZIP_Code', 'Population']]
crimes_integrated = pd.merge(crimes_with_neigh, valid_pop, on='ZIP_Code', how='left')

In [18]:
crimes_by_neigh = crimes_integrated.groupby('Neighborhood').agg({
    'Offense Case Type': 'count',  # crime nums 
    'Population': 'mean',
    'Race': lambda x: x.value_counts().to_dict() if not x.empty else {}  # race distribution
}).reset_index()
crimes_by_neigh.rename(columns={'Offense Case Type': 'Crime_Count'}, inplace=True)
crimes_by_neigh['Crime_Rate_per_1000'] = (crimes_by_neigh['Crime_Count'] / crimes_by_neigh['Population']) * 1000
crimes_by_neigh = crimes_by_neigh.sort_values('Crime_Rate_per_1000', ascending=False)

In [19]:
#Final Storage
crimes_integrated.to_csv('final_integrated_crimes.csv', index=False)
crimes_by_neigh.to_csv('analysis_by_neighborhood.csv', index=False)

##First Match

In [20]:
print("Integration is Done !")
print("Match rate:", match_rate, "%")
print("\  Neighborhoods with highest rate:")
print(crimes_by_neigh.head(10))

Integration is Done !
Match rate: 1.9323013669747338 %
\  Neighborhoods with highest rate:
              Neighborhood  Crime_Count    Population  \
6          Downtown Austin          345  11625.000000   
8              East Austin          387  23556.532300   
15            North Austin          475  45596.637895   
24            South Austin          259  35019.069498   
3               Cherrywood           48   6618.000000   
23                Rosedale           61   8426.000000   
0     Allandale, Crestview          158  23847.000000   
29  Tarrytown, Clarksville          134  22194.000000   
30            Wells Branch          131  25555.000000   
11               Hyde Park           83  17071.000000   

                                                 Race  Crime_Rate_per_1000  
6   {'White': 117, 'Black': 20, 'WHITE': 9, 'Middl...            29.677419  
8   {'White': 165, 'Black': 34, 'WHITE': 18, 'Asia...            16.428564  
15  {'White': 178, 'Black': 16, 'WHITE': 14, 'Asia